In [1]:
import pandas as pd
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import re

In [45]:
scenario_folder = '1km_all_scenarios_dr5'
scenario = 'WWR per cluster'
df = pd.read_csv(os.path.join(scenario_folder,
                              scenario,
                              f'{scenario}.gz'))

In [46]:
df['IrrigatedArea'].sum()

419857.0000000001

In [47]:
dff_water = df.groupby('Cluster').agg({'IrrigationWaterAverage': 'sum', 'PopulationWaterAverage': 'sum', 'FinalAverageIrrigationWater': 'sum',
                                       'IrrigationAverageReusedWater': 'sum', 'PopulationAverageReusedWater': 'sum'})

In [48]:
dff_water.reset_index(inplace=True)

In [49]:
fig = go.Figure()
dff = dff_water

dff['total_extractions'] = dff['FinalAverageIrrigationWater'] + dff['PopulationWaterAverage']
dff['cat_order'] = [cluster for value, cluster in dff[['total_extractions', 'Cluster']].sort_values('total_extractions', ascending=True).values]

cat_order = dff['cat_order'].copy()

fig.add_trace(go.Bar(y=dff['Cluster'], x=dff['PopulationWaterAverage']/1000000, orientation='h', name='Population water extractions', marker=dict(color='#fdb863', line=dict(width=0, color='Grey'))))
fig.add_trace(go.Bar(y=dff['Cluster'], x=dff['FinalAverageIrrigationWater']/1000000, orientation='h', name='Irrigation water extractions', marker=dict(color='#e66101', line=dict(width=0, color='Grey'))))
fig.add_trace(go.Bar(y=dff['Cluster'], x=-dff['PopulationAverageReusedWater']/1000000, orientation='h', name='Resued water from population', marker=dict(color='#5e3c99', line=dict(width=0, color='Grey'))))
fig.add_trace(go.Bar(y=dff['Cluster'], x=-dff['IrrigationAverageReusedWater']/1000000, orientation='h', name='Resued water from irrigation', marker=dict(color='#b2abd2', line=dict(width=0, color='Grey'))))

fig.update_layout(barmode='relative', height=700, template='plotly_white',
                  yaxis={'categoryorder': 'array', 'categoryarray': cat_order,
                         'title': 'Cluster', 'type': 'category', 'ticks': "outside"}, 
                  xaxis={'title': 'Million cubic meters of water per year (Mm<sup>3</sup>/yr)'},
                  legend=dict(
                            orientation="h",
                            yanchor="bottom",
                            y=1.02,
                            xanchor="left",
                            x=0
                ),
                 margin=dict(b=0, l=0, r=0, t=0))
fig.show()
# fig.write_image('BaselineWater2.pdf')

In [63]:
list(df)

['X',
 'Y',
 'CI',
 'GroundwaterDepth',
 'IrrigatedArea',
 'Population',
 'Region',
 'TDS',
 'geometry',
 'prec_1',
 'prec_2',
 'prec_3',
 'prec_4',
 'prec_5',
 'prec_6',
 'prec_7',
 'prec_8',
 'prec_9',
 'prec_10',
 'prec_11',
 'prec_12',
 'wind_1',
 'wind_2',
 'wind_3',
 'wind_4',
 'wind_5',
 'wind_6',
 'wind_7',
 'wind_8',
 'wind_9',
 'wind_10',
 'wind_11',
 'wind_12',
 'tmin_1',
 'tmin_2',
 'tmin_3',
 'tmin_4',
 'tmin_5',
 'tmin_6',
 'tmin_7',
 'tmin_8',
 'tmin_9',
 'tmin_10',
 'tmin_11',
 'tmin_12',
 'tmax_1',
 'tmax_2',
 'tmax_3',
 'tmax_4',
 'tmax_5',
 'tmax_6',
 'tmax_7',
 'tmax_8',
 'tmax_9',
 'tmax_10',
 'tmax_11',
 'tmax_12',
 'tavg_1',
 'tavg_2',
 'tavg_3',
 'tavg_4',
 'tavg_5',
 'tavg_6',
 'tavg_7',
 'tavg_8',
 'tavg_9',
 'tavg_10',
 'tavg_11',
 'tavg_12',
 'elevation',
 'lat',
 'long',
 'srad_1',
 'srad_2',
 'srad_3',
 'srad_4',
 'srad_5',
 'srad_6',
 'srad_7',
 'srad_8',
 'srad_9',
 'srad_10',
 'srad_11',
 'srad_12',
 'ISO',
 'Province',
 'crop_share',
 'ETo_1',
 'ETo_2'

In [65]:
dff_energy = df.groupby('Cluster').agg({'FinalAveragePumpingEnergy': 'sum',
                                        'FinalAverageTreatmentEnergy': 'mean', 'FinalAverageDesalinationEnergy': 'sum',
                                        'TDS': 'mean', 'GroundwaterDepth': 'mean'})

In [66]:
dff_energy.reset_index(inplace=True)

In [69]:
fig = go.Figure()
dff = dff_energy

fig.add_trace(go.Bar(x=dff['Cluster'], y=dff['FinalAveragePumpingEnergy']/1000000, name='Pumping Energy', marker=dict(color='#fc8d62', line=dict(width=1, color='Grey'))))
fig.add_trace(go.Bar(x=dff['Cluster'], y=dff['FinalAverageDesalinationEnergy']/1000000, name='Desalination Energy', marker=dict(color='#66c2a5', line=dict(width=1, color='Grey'))))
fig.add_trace(go.Bar(x=dff['Cluster'], y=dff['FinalAverageTreatmentEnergy']/1000000, name='Treatment Energy', marker=dict(color='#8da0cb', line=dict(width=1, color='Grey'))))

fig.update_layout(barmode='stack', width=900, template='plotly_white',
                  xaxis={'categoryorder': 'sum descending', #'categoryarray': cat_order.iloc[::-1],
                         'title': 'Cluster', 'type': 'category', 'ticks': "outside"}, 
                  yaxis={'title': 'Energy (GWh/yr)'},
                  legend=dict(
                            orientation="h",
                            yanchor="bottom",
                            y=1.02,
                            xanchor="right",
                            x=1
                ),
                 margin=dict(b=0, l=0, r=0, t=0))
fig.show()
# fig.write_image('BaselineEnergy.pdf')

In [10]:
scenarios = np.array([x[1] for x in os.walk('.')][0])

In [36]:
scenarios = np.delete(scenarios, 0)

In [37]:
for scenario in scenarios:
    df = pd.read_csv(os.path.join(scenario, scenario.replace('agri','ag') + '.gz'))
    dff = df.groupby('Cluster').agg({'FinalPumpingEnergy': 'sum', 'FinalDesalinationEnergy': 'sum', 'FinalTreatmentEnergy': 'first'})
    dff.reset_index(inplace=True)
    dff['Cluster'] = dff['Cluster'].astype(int).astype(str)
    
    fig = go.Figure()

    fig.add_trace(go.Bar(x=dff['Cluster'], y=dff['FinalTreatmentEnergy']/1000000, name='Treatment Energy', 
                         marker=dict(color='#8da0cb', line=dict(width=1, color='Grey'))))
    fig.add_trace(go.Bar(x=dff['Cluster'], y=dff['FinalPumpingEnergy']/1000000, name='Pumping Energy', 
                         marker=dict(color='#fc8d62', line=dict(width=1, color='Grey'))))
    fig.add_trace(go.Bar(x=dff['Cluster'], y=dff['FinalDesalinationEnergy']/1000000, name='Desalination Energy', 
                         marker=dict(color='#66c2a5', line=dict(width=1, color='Grey'))))

    fig.update_layout(barmode='stack', width=900, template='plotly_white',
                      xaxis={'categoryorder':'sum descending', 'title': 'Cluster', 'type': 'category', 'ticks': "outside"}, 
                      yaxis={'title': 'Energy (GWh/yr)'},
                      legend=dict(
                            orientation="h",
                            yanchor="bottom",
                            y=1.02,
                            xanchor="right",
                            x=1
                            ),
                     margin=dict(b=0, l=0, r=0, t=0))
    fig.write_image(os.path.join(scenario,'EnergyPerCluster2.pdf'))

In [73]:
fig = go.Figure()
dff = dff_energy.merge(dff_water)


fig.add_trace(go.Scatter(x=dff[['IrrigationWaterAverage','PopulationWaterAverage']].sum(axis=1)/1000000, 
                         y=dff[['FinalAveragePumpingEnergy','FinalAverageDesalinationEnergy', 'FinalAverageTreatmentEnergy']].sum(axis=1)/1000000,
                         mode='markers',
                         text='Cluster: ' + dff['Cluster'].astype(int).astype(str),
                         marker=dict(size = dff['TDS']/dff['TDS'].min()*10,
                                    color=dff['GroundwaterDepth'],
                                    colorscale='viridis',
                                    showscale=True
                        )))
# fig.add_trace(go.Bar(x=dff['Cluster'], y=dff['IrrigationDesalinationEnergy']/1000000, name='Desalination Energy', marker=dict(color='#66c2a5', line=dict(width=1, color='Grey'))))

fig.update_layout(width=900, template='plotly_white',
                  xaxis={'categoryorder':'sum descending', 'title': 'Water (Mm<sup>3</sup>/yr)'}, 
                  yaxis={'title': 'Energy (GWh/yr)'},
                  legend=dict(
                            orientation="h",
                            yanchor="bottom",
                            y=1.02,
                            xanchor="right",
                            x=1
                ),
                 margin=dict(b=0, l=0, r=0, t=0))
fig.show()
# fig.write_image('BaselineEnergy.pdf')

In [72]:
dff.to_csv(os.path.join(scenario_folder,
                        scenario,
                        'DataForNexusPlot.csv'))

In [11]:
dff = df.groupby('Cluster').agg({'MovingBedBiofilmReactorLCOW': 'mean', 'TricklingFilterLCOW': 'mean', 'IntermittentSandFilterLCOW': 'mean', 
                             'RotatingBiologicalContractorsLCOW': 'mean', 'SequencingBatchReactorLCOW': 'mean', 'Membrane BioreactorLCOW': 'mean', 
                             'ExtendedAerationLCOW': 'mean', 'PopulationWaterAverage': 'sum','FinalAverageIrrigationWater': 'sum', 'PondSystemLCOW': 'mean'}).reset_index()

In [12]:
dff['total_extractions'] = dff['FinalAverageIrrigationWater'] + dff['PopulationWaterAverage']
dff['cat_order'] = [cluster for value, cluster in dff[['total_extractions', 'Cluster']].sort_values('total_extractions', ascending=False).values]

dff_sorted = dff.sort_values('total_extractions', ascending=False).reset_index(drop=True)
dff_sorted = dff_sorted.loc[range(10)]

In [13]:
dff_melt = dff_sorted.melt(id_vars=['Cluster', 'PopulationWaterAverage', 
                             'FinalAverageIrrigationWater', 'total_extractions', 'cat_order'])

In [14]:
from plotly.subplots import make_subplots

dff_melt = dff_sorted.melt(id_vars=['Cluster', 'PopulationWaterAverage', 
                             'FinalAverageIrrigationWater', 'total_extractions', 'cat_order'])

dff_melt['variable'] = [' '.join(re.findall('[A-Z][^A-Z]*', s.replace('LCOW', ''))).capitalize() for s in dff_melt['variable']]
dff_melt.sort_values('value', ascending=False, inplace=True)
dff_melt['FinalAverageIrrigationWater'] /= 1000000
dff_melt['PopulationWaterAverage'] /= 1000000
dff_melt['PopColor'] = 'Population water extractions'
dff_melt['AgColor'] = 'Irrigation water extractions'

# fig = make_subplots(specs=[[{"secondary_y": True}]])
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03)

data = px.bar(dff_melt.sort_values('value'), y='value', x='Cluster', color='variable',
           color_discrete_sequence=px.colors.qualitative.T10).data
for trace in data:
    fig.add_trace(
        trace, 
        row=2, col=1
    )


fig.add_trace(
    px.scatter(dff_melt, y='PopulationWaterAverage', x='Cluster', color='PopColor',
              color_discrete_sequence=[px.colors.qualitative.T10[9]],
              ).data[0],
#     secondary_y=True,
    row=1, col=1, 
)

fig.add_trace(
    px.scatter(dff_melt, y='FinalAverageIrrigationWater', x='Cluster', color='AgColor',
              color_discrete_sequence=['LightSkyBlue']
              ).data[0],
    row=1, col=1
#     secondary_y=True,
)

fig.update_xaxes(type='category')
fig.update_layout(width=900, template='plotly_white', barmode='group',
                  legend=dict(
                            orientation="h",
                            yanchor="top",
                            y=-0.15,
                            xanchor="center",
                            x=0.5
                ),
                 margin=dict(b=0, l=0, r=0, t=30))

fig.update_traces(marker=dict(size=10,
#                               symbol='octagon',
#                               line=dict(width=1,
#                                         color='DarkSlateGrey')
                             ),
                  selector=dict(mode='markers'))
fig.update_xaxes(title='Cluster', showticklabels=True, categoryorder='array', 
                 categoryarray=dff['cat_order'].loc[dff['cat_order'].isin(dff_melt['Cluster'])],
                 col=1, row=2)
                         
fig.update_yaxes(title_text="Water extractions (Mm<sup>3</sup>)", type='log', col=1, row=1)
fig.update_yaxes(title_text="LCOW ($/m<sup>3</sup>)", col=1, row=2)

fig.add_annotation(text="<b>a)</b>",
                  xref="paper", yref="paper",
                  x=-0.061, y=1.09, showarrow=False)

fig.add_annotation(text="<b>b)</b>",
                  xref="paper", yref="paper",
                  x=-0.061, y=0.45, showarrow=False)

fig.show()
# fig.write_image('../LCOW_largest_clusters.pdf')

In [15]:
dff = df.groupby('Cluster').agg({'MovingBedBiofilmReactorLCOW': 'mean', 'TricklingFilterLCOW': 'mean', 'IntermittentSandFilterLCOW': 'mean', 
                             'RotatingBiologicalContractorsLCOW': 'mean', 'SequencingBatchReactorLCOW': 'mean', 'Membrane BioreactorLCOW': 'mean', 
                             'ExtendedAerationLCOW': 'mean', 'PopulationAverageReclaimedWater': 'mean','IrrigationAverageReclaimedWater': 'mean', 'PondSystemLCOW': 'mean'}).reset_index()
dff_melt = dff.melt(id_vars=['Cluster', 'PopulationAverageReclaimedWater', 
                             'IrrigationAverageReclaimedWater', 'PondSystemLCOW'])
dff_melt = dff_melt.loc[dff_melt['value']<=1.02]
dff_melt['variable'] = [' '.join(re.findall('[A-Z][^A-Z]*', s.replace('LCOW', ''))).capitalize() for s in dff_melt['variable']]

dff_melt['PopulationAverageReclaimedWater'] /= 1000000

colors = [px.colors.qualitative.T10[i] for i in [0, 1, 3, 4, 5, 6, 7]]

fig_pop_all = px.scatter(dff_melt.sort_values('value'), x='PopulationAverageReclaimedWater', 
                 y='value', color='variable', text='Cluster',
#                  marginal_x='rug', marginal_y='box',
                 color_discrete_sequence=colors,
                )
# for i in range(0, 21, 3):              
#     fig.data[i].update(mode='lines+markers', opacity=0.8)

# x = np.array(fig.data)
# fig.data = tuple(np.delete(x, [4, 7, 10, 13, 16, 19]))
# fig.data = tuple(np.delete(x, [3, 5, 7, 9, 11, 13]))
# fig.data[1].update(marker={'color': px.colors.qualitative.T10[9]}, xbins=dict(size=2000000))

fig_pop_all.update_traces(mode='lines+markers', opacity=0.8)

# for x, cluster in zip(dff_melt['PopulationFutureReclaimedWater'].unique(), dff_melt['Cluster'].unique()):
#     fig.add_annotation(yref="paper", x=x, y=0.8,
#                       text=cluster,
#                       showarrow=True,
# #                      arrowhead=1
#                       )
    
fig_pop_all.update_layout(width=900, template='plotly_white',
                  xaxis={'title': 'Capacity (Mm<sup>3</sup>/y)'}, 
                  yaxis={'title': 'LCOW ($/m<sup>3</sup>)'},
                  legend=dict(
#                             title='',
                            orientation="h",
                            yanchor="top",
                            y=-0.15,
                            xanchor="center",
                            x=0.5
                ),
                 margin=dict(b=0, l=0, r=0, t=30))    

fig_pop_all.show()
# fig_pop_all.write_image('../LCOW_population_all2.pdf')

In [16]:
dff_least = df.groupby(['Cluster', 'PopulationLeastCostTechnology', 'PopulationAverageReclaimedWater'])[['PopulationLeastCost']].mean().reset_index()

colors = [px.colors.qualitative.T10[i] for i in [0, 1, 5]]

class_name_pop = {0: 'NaN', 1: 'Extended aeration', 2: 'Membrane bioreactor',
                  3: 'Sequencing batch reactor', 4: 'Rotating biological contractors',
                  5: 'Intermittent sand filter', 6: 'Trickling filter',
                  7: 'Moving bed biofilm reactor'}

dff_least['PopulationLeastCostTechnology'] = [class_name_pop[tech] for tech in dff_least['PopulationLeastCostTechnology']]
dff_least['PopulationAverageReclaimedWater'] /= 1000000    

fig_pop = make_subplots(rows=1, cols=2)
    
fig_pop = px.line(dff_least.sort_values('PopulationLeastCost'), x='PopulationAverageReclaimedWater', 
                 y='PopulationLeastCost', color='PopulationLeastCostTechnology', text='Cluster',
#                  marginal_x='rug', marginal_y='box',
                 color_discrete_sequence=colors, 
                )
# for i in range(0, 21, 3):              
#     fig.data[i].update(mode='lines+markers', opacity=0.8)

# x = np.array(fig.data)
# fig.data = tuple(np.delete(x, [4, 7, 10, 13, 16, 19]))
# fig.data = tuple(np.delete(x, [3, 5, 7, 9, 11, 13]))
# fig.data[1].update(marker={'color': px.colors.qualitative.T10[9]}, xbins=dict(size=2000000))


fig_pop.update_traces(mode='lines+markers', opacity=0.8, showlegend=False)

# for x, cluster in zip(dff_melt['PopulationFutureReclaimedWater'].unique(), dff_melt['Cluster'].unique()):
#     fig.add_annotation(yref="paper", x=x, y=0.8,
#                       text=cluster,
#                       showarrow=True,
# #                      arrowhead=1
#                       )
    
fig_pop.update_layout(width=900, template='plotly_white',
                  xaxis={'title': 'Capacity (Mm<sup>3</sup>/y)'}, 
                  yaxis={'title': 'LCOW ($/m<sup>3</sup>)'},
                  legend=dict(
#                             title='',
                            orientation="h",
                            yanchor="top",
                            y=-0.15,
                            xanchor="center",
                            x=0.5
                ),
                 margin=dict(b=0, l=0, r=0, t=30))    

fig_pop.show()
# fig_pop.write_image('../LCOW_population_least_cost.pdf')

In [17]:
capex = '(3897.7*(v*1500/400/365)**(-0.407))*(v*1500/400/365)'
opex = '5.543*(v*1500/400/365)+3127.5'

In [43]:
v = np.arange(800000., 400000000., 1000000)
y = np.arange(0, 36)
r = 0.04
lcow_capex = eval(capex) / (v * sum((1 / (1 + r))**y))
lcow_opex = eval(opex) / (v)
lcow = lcow_capex + lcow_opex

In [44]:
dff_ponds = dff.rename(columns={'PondSystemLCOW': 'Pond system'})
dff_ponds = dff_ponds.sort_values('IrrigationAverageReclaimedWater', ascending=False)
dff_ponds['IrrigationAverageReclaimedWater'] /= 1000000
dff_ponds.dropna(inplace=True)

fig_ag = go.Figure()

fig_ag.add_scatter(y=lcow, x=v/1000000,
             mode='lines',
             marker={'color': px.colors.qualitative.T10[2]},
             name='Pond system LCOW',
             opacity=1)

fig_ag.add_scatter(x=dff_ponds['IrrigationAverageReclaimedWater'], 
                   y=dff_ponds['Pond system'],
                   mode='markers+text',
                   text=['']*40,
                   marker={'color': px.colors.qualitative.T10[2], 'opacity': 0.6},
                   name='Clusters',
                   )


arrows = {0: (20, -5), 1: (10, -20), 2: (10, -20), 3: (5, -20), 4: (-15, 10),
          5: (20, 5), 6: (-15, -10), 7: (-10, 20), 8: (10, 20), 9: (20, -20),
          10: (40, -40), 11: (10, -20), 12: (-30, 25), 13: (10, -20), 14: (10, -20),
          15: (10, -20), 16: (15, -20), 17: (40, -50), 18: (30, -30), 19: (-30, 10),
          20: (-15, 25), 21: (10, -20), 22: (-25, -20), 23: (25, -25), 24: (-10, -20),
          25: (18, -20), 26: (5, 20), 27: (25, -35), 28: (-20, -10), 29: (-15, 30),
          30: (18, 20), 31: (5, 20), 32: (-35, 1), 33: (-20, -15), 34: (20, -40),
          35: (-25, -5), 36: (-20, -20), 37: (-28, 18), 38: (10, -20), 39: (-5, 20),}

for i, row,  in dff_ponds.reset_index().iterrows(): 
    fig_ag.add_annotation(
            x=row['IrrigationAverageReclaimedWater'],
            y=row['Pond system'],
            xref="x",
            yref="y",
            text=int(row['Cluster']),
            showarrow=True,
            font=dict(
                size=10,
                ),
            align="center",
#             standoff=3,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=1,
#             arrowtail=0,
            arrowside='none',
            arrowcolor="#636363",
            xshift=np.sign(arrows[int(row['Cluster'])][0]) * 3 / (1 + (arrows[int(row['Cluster'])][1]/arrows[int(row['Cluster'])][0])**2)**(1/2),
            yshift=-np.sign(arrows[int(row['Cluster'])][1]) * 3 / (1 + (arrows[int(row['Cluster'])][0]/arrows[int(row['Cluster'])][1])**2)**(1/2),
            ax=arrows[int(row['Cluster'])][0],
            ay=arrows[int(row['Cluster'])][1],
            opacity=0.5,
    )
    
fig_ag.update_layout(width=500, template='plotly_white',
                  xaxis={'title': 'Capacity (Mm<sup>3</sup>/y)'}, 
                  yaxis={'title': 'LCOW ($/m<sup>3</sup>)'},
                  legend=dict(
#                             title='',
                            orientation="h",
                            yanchor="top",
                            y=-0.15,
                            xanchor="center",
                            x=0.5
                ),
                 margin=dict(b=0, l=0, r=0, t=30))    

fig_ag.show()
# fig_ag.write_image('../LCOW_agriculture.pdf')

In [23]:
fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.03, shared_yaxes=True
#                    specs=[[{"colspan": 2}, None], 
#                           [{}, {}]]
                   )
# for trace in fig_ag.data:
#     fig.add_trace(trace, col=1, row=1)
for trace in fig_pop_all.data:
    fig.add_trace(trace, col=1, row=1)
for trace in fig_pop.data:
    fig.add_trace(trace, col=2, row=1)
    
fig.add_trace(go.Scatter(x=[10], y=[2], mode='markers+text', text='', 
                         marker=dict(color='#636363', opacity=0.5), name='Clusters'))

arrows = {0: (10, -20), 1: (5, -20), 2: (10, -20), 3: (5, -20), 4: (-25, 5),
          5: (-20, 30), 6: (-30, 20), 7: (10, 20), 8: (-30, 10), 9: (-20, -20),
          10: (-30, 30), 11: (10, -20), 12: (-5, 25), 13: (10, -20), 14: (20, -30),
          15: (10, -20), 16: (15, -20), 17: (10, -45), 18: (20, -40), 19: (-20, 40),
          20: (10, 25), 21: (10, -20), 22: (20, -40), 23: (25, -25), 24: (5, -20),
          25: (18, -20), 26: (1, 20), 27: (10, -35), 28: (-20, -10), 29: (-15, 30),
          30: (-5, 20), 31: (5, 20), 32: (-5, 20), 33: (-20, -15), 34: (20, -40),
          35: (10, -20), 36: (-15, 20), 37: (-20, 18), 38: (20, -20), 39: (-20, 1),}

for i, row,  in dff_least.iterrows():        
    fig.add_annotation(
            x=row['PopulationAverageReclaimedWater'],
            y=row['PopulationLeastCost'],
            xref="x",
            yref="y",
            text=int(row['Cluster']),
            showarrow=True,
            font=dict(
                size=10,
                ),
            align="center",
            arrowside='none',
            arrowhead=1,
            arrowsize=1,
            arrowwidth=1,
            arrowcolor="#636363",
            ax=arrows[int(row['Cluster'])][0],
            ay=arrows[int(row['Cluster'])][1],
            xshift=np.sign(arrows[int(row['Cluster'])][0]) * 3 / (1 + (arrows[int(row['Cluster'])][1]/arrows[int(row['Cluster'])][0])**2)**(1/2),
            yshift=-np.sign(arrows[int(row['Cluster'])][1]) * 3 / (1 + (arrows[int(row['Cluster'])][0]/arrows[int(row['Cluster'])][1])**2)**(1/2),
            opacity=0.5,
            col=2,
            row=1
    )
    
fig.add_annotation(text="<b>a)</b>",
                  xref="paper", yref="paper",
                  x=-0.04, y=-0.1, showarrow=False)

fig.add_annotation(text="<b>b)</b>",
                  xref="paper", yref="paper",
                  x=0.5, y=-0.1, showarrow=False)
    
    
fig.update_layout(width=900, template='plotly_white',
                  yaxis={'title': 'LCOW ($/m<sup>3</sup>)'},
                  legend=dict(
                            title='',
                            orientation="h",
                            yanchor="top",
                            y=-0.15,
                            xanchor="center",
                            x=0.5
                ),
                 margin=dict(b=0, l=0, r=0, t=30))  

fig.update_xaxes(title_text='Capacity (Mm<sup>3</sup>/y)')
fig.update_yaxes(title_text="LCOW ($/m<sup>3</sup>)", col=1, range=(0,0.3))
    
fig.show()
# fig.write_image('../LCOW_pop_ag.pdf')